In [ ]:
import sys
import os
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src.config import DATA_PATH, PFF_PROP_PATH


pd.set_option('display.max_columns', None)  # Display all columns

In [ ]:
import importlib

importlib.reload(utils)

In [ ]:
year = 2023
week = 3

In [ ]:
# Read in PFF Prop Data
pff_df = pd.read_csv(PFF_PROP_PATH /f'nfl-best-bets-2023-week-{week}.csv')
pff_df.shape

In [ ]:
pff_df.head()

In [ ]:
edges = []
overs = []
implied = []

for row in pff_df.itertuples():
    side = 'One' if row.sideOneValue > row.sideTwoValue else 'Two'

    # Use getattr to access column values dynamically
    side_value = getattr(row, f'side{side}Value')
    side_type = getattr(row, f'side{side}Type')
    side_odds = getattr(row, f'side{side}Odds')

    edges.append(side_value)
    overs.append(True if side_type == 'over' else False)
    implied.append(utils.implied_probability(side_odds))

pff_df['pff_edge'] = edges
pff_df['over_bet'] = overs
pff_df['implied_prob'] = implied

In [ ]:
pff_df.head()

In [ ]:
pff_df = pff_df[['propType', 'player', 'position', 'team', 'opponent', 'line', 'pff_edge', 'over_bet', 'implied_prob']]

In [ ]:
for team_col in ['team', 'opponent']:
    pff_df[team_col] = pff_df[team_col].apply(utils.standardize_teams)

pff_df['week'] = week
pff_df['player'] = pff_df['player'].str.replace(r'\s+', '', regex=True)


In [ ]:
pff_df = pff_df[~pff_df['propType'].str.contains('game')]
pff_df.shape

In [ ]:
weekly_df = nfl.import_weekly_data([year])

In [ ]:
weekly_df.head()

In [ ]:
weekly_df.columns

In [ ]:
weekly_df = weekly_df[weekly_df['week'] == week]
weekly_df = weekly_df[[
    'player_name',
    'position',
    'position_group', 
    'recent_team',
    'opponent_team', 
    'completions', 
    'attempts',
    'passing_yards', 
    'passing_tds', 
    'interceptions',
    'carries',
    'rushing_yards',
    'receptions', 
    'targets', 
    'receiving_yards',
    'target_share',
]]
weekly_df.head()

In [ ]:
df = pd.merge(pff_df, weekly_df, how='left', left_on=['player', 'position', 'team'], right_on=['player_name', 'position', 'recent_team'])

In [ ]:
df.shape

In [ ]:
df['rush_recv_yd'] = df['rushing_yards'] + df['receiving_yards']

In [ ]:
df.sort_values('pff_edge').head()

In [ ]:
# remap columns to prop categories
df = df.rename(columns={
    'attempts': 'pass_att',
    'completions': 'pass_comp',
    'passing_yards': 'pass_yd',
    'passing_tds': 'pass_td',
    'interceptions': 'pass_int',
    'receptions': 'recv_rec',
    'receiving_yards': 'recv_yd',
    'carries': 'rush_att',
    'rushing_yards': 'rush_yd',
})

In [ ]:
df.head()

In [ ]:
winners = []

for row in df.itertuples():
    res = False
    if getattr(row, 'over_bet'):
        res = getattr(row, getattr(row, 'propType')) > getattr(row, 'line')
    else:
        res = getattr(row, getattr(row, 'propType')) < getattr(row, 'line')
    winners.append(res)

df['win'] = winners

In [ ]:
df.head()

In [ ]:
df.win.mean()

In [ ]:
df[df['pff_edge'] > 0.02].win.mean()

In [ ]:
df['decimal_odds'] = df.implied_prob.apply(utils.decimal_odds)
df['profit'] = df['win'] * df['decimal_odds']

In [ ]:
df[df['pff_edge'] > 0.05].profit.mean()

In [ ]:
df[df['pff_edge'] > 0.20].profit.mean()

In [ ]:
df.head(25)